In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd

## 1. Attendance Data

- Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
# Renaming first column for readability 
attendance = attendance.rename(columns={'Unnamed: 0':'name'})
attendance.head()

,name,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
# Melt data
attendance_melted = attendance.melt(id_vars= 'name', var_name='date', value_name = 'attendance')
attendance_melted.head()

,name,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [5]:
# Create copy of attendance column then replace letters with weighted values
attendance_melted['value'] = attendance_melted.attendance
attendance_melted = attendance_melted.replace({'value': {'P':1, 'A':0, 'T':.9, 'H':.5}})
attendance_melted.head()

,name,date,attendance,value
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [6]:
#Calculate attendance percentages for each student
attendance_melted.groupby('name').value.mean()

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: value, dtype: float64

## 2. Coffee Levels
- Read the coffee_levels table.

In [7]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


- Transform the data so that each carafe is in it's own column.

In [8]:
#Exploring
coffee.coffee_carafe.value_counts()

x    10
y    10
z    10
Name: coffee_carafe, dtype: int64

In [9]:
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


In [10]:
coffee_tidy = coffee.pivot_table(index='hour', columns='coffee_carafe', values='coffee_amount').reset_index()
coffee_tidy.columns.name = ''
coffee_tidy.head()

,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


- Is this the best shape for the data?

My Answer: I think this is a legible shape to read the data, but wouldn't say its best.

## 3. Cake Recipes
- Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [11]:
cake = pd.read_csv('untidy-data/cake_recipes.csv')
cake.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


- Tidy the data as necessary.

In [12]:
#Splitting the recipe:position columns into two and dropping it
cake[['recipe', 'position']] = cake['recipe:position'].str.split(':', expand=True)
cake = cake.drop(columns='recipe:position')
cake.head()

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom


In [13]:
#Reshape the data with melt
cake_tidy = cake.melt(id_vars=['recipe', 'position'], var_name='temperature', value_name='tastiness')
cake_tidy.head()

,recipe,position,temperature,tastiness
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [14]:
# Created a pivot table for practice
cake_pivot = cake_tidy.pivot_table(index=['recipe', 'position'], columns='temperature', values='tastiness')
cake_pivot

temperature            225        250        275        300
recipe position                                            
a      bottom    61.738655  53.912627  74.414730  98.786784
       top       51.709751  52.009735  68.576858  50.228470
b      bottom    57.095320  61.904369  61.196980  99.248541
       top       82.455004  95.224151  98.594881  58.169349
c      bottom    96.470207  52.001358  92.893227  65.473084
       top       71.306308  82.795477  92.098049  53.960273
d      bottom    52.799753  58.670419  51.747686  56.183110
       top       96.873178  76.101363  59.571620  50.971626

- Which recipe, on average, is the best? recipe b

In [15]:
cake_tidy.groupby('recipe').mean()

,tastiness
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


- Which oven temperature, on average, produces the best results? 275

In [16]:
cake_tidy.groupby('temperature').mean()

,tastiness
temperature,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


- Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [17]:
cake_tidy.sort_values(by='tastiness', ascending=False).head(1)

,recipe,position,temperature,tastiness
26,b,bottom,300,99.248541


## Bonus - Dem Score, Pew

**Dem Score Exploration**

In [18]:
#Reading Data
df = pd.read_csv('untidy-data/dem_score.csv')
df.head()

,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  96 non-null     object
 1   1952     96 non-null     int64 
 2   1957     96 non-null     int64 
 3   1962     96 non-null     int64 
 4   1967     96 non-null     int64 
 5   1972     96 non-null     int64 
 6   1977     96 non-null     int64 
 7   1982     96 non-null     int64 
 8   1987     96 non-null     int64 
 9   1992     96 non-null     int64 
dtypes: int64(9), object(1)
memory usage: 7.6+ KB


In [20]:
df_tidy = df.melt(id_vars= 'country', var_name='year', value_name = 'democracy_score')
df_tidy.head()

,country,year,democracy_score
0,Albania,1952,-9
1,Argentina,1952,-9
2,Armenia,1952,-9
3,Australia,1952,10
4,Austria,1952,10


In [21]:
#Top 10 Highest scoring countries
df_tidy.sort_values(by='democracy_score', ascending=False).head(10)

,country,year,democracy_score
541,Netherlands,1977,10
675,Australia,1987,10
667,United Kingdom,1982,10
136,Ireland,1957,10
137,Israel,1957,10
138,Italy,1957,10
139,Japan,1957,10
668,United States,1982,10
795,Finland,1992,10
352,Norway,1967,10


In [22]:
# 10 Lowest Scoring Countries
df_tidy.sort_values(by='democracy_score').head(10)

,country,year,democracy_score
555,Saudi Arabia,1977,-10
584,Bhutan,1982,-10
75,Saudi Arabia,1952,-10
230,Iran,1962,-10
363,Saudi Arabia,1967,-10
545,Oman,1977,-10
257,Oman,1962,-10
524,Jordan,1977,-10
518,Iran,1977,-10
488,Bhutan,1977,-10


**Pew Exploration**

In [23]:
#Reading Data
df = pd.read_csv('untidy-data/pew.csv')
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   religion            18 non-null     object
 1   <$10k               18 non-null     int64 
 2   $10-20k             18 non-null     int64 
 3   $20-30k             18 non-null     int64 
 4   $30-40k             18 non-null     int64 
 5   $40-50k             18 non-null     int64 
 6   $50-75k             18 non-null     int64 
 7   $75-100k            18 non-null     int64 
 8   $100-150k           18 non-null     int64 
 9   >150k               18 non-null     int64 
 10  Don't know/refused  18 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.7+ KB


In [25]:
df.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
religion,Agnostic,Atheist,Buddhist,Catholic,Don’t know/refused,Evangelical Prot,Hindu,Historically Black Prot,Jehovah's Witness,Jewish,Mainline Prot,Mormon,Muslim,Orthodox,Other Christian,Other Faiths,Other World Religions,Unaffiliated
<$10k,27,12,27,418,15,575,1,228,20,19,289,29,6,13,9,20,5,217
$10-20k,34,27,21,617,14,869,9,244,27,19,495,40,7,17,7,33,2,299
$20-30k,60,37,30,732,15,1064,7,236,24,25,619,48,9,23,11,40,3,374
$30-40k,81,52,34,670,11,982,9,238,24,25,655,51,10,32,13,46,4,365
$40-50k,76,35,33,638,10,881,11,197,21,30,651,56,9,32,13,49,2,341
$50-75k,137,70,58,1116,35,1486,34,223,30,95,1107,112,23,47,14,63,7,528
$75-100k,122,73,62,949,21,949,47,131,15,69,939,85,16,38,18,46,3,407
$100-150k,109,59,39,792,17,723,48,81,11,87,753,49,8,42,14,40,4,321
>150k,84,74,53,633,18,414,54,78,6,151,634,42,6,46,12,41,4,258


In [26]:
df_tidy = df.melt(id_vars='religion', var_name='income_bracket', value_name='respondents')
df_tidy.head()

,religion,income_bracket,respondents
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


In [27]:
df_tidy.groupby('religion').mean()

,respondents
religion,
Agnostic,82.6
Atheist,51.5
Buddhist,41.1
Catholic,805.4
Don’t know/refused,27.2
Evangelical Prot,947.2
Hindu,25.7
Historically Black Prot,199.5
Jehovah's Witness,21.5
